Import repo and funcs

In [2]:
import torch

Import function

In [3]:
def get_data(path):
    with open(path, 'r') as f:
        data = f.readlines()[3:]
        data = [float(data[i].strip()) for i in range(len(data))]
    return data

In [4]:
def read_data(din, dout, d1, d2, l1, l2):
    data = []
    for i in range(d1):
        data.append(get_data(r'D:\\My Files\\核技术专题研讨\\n-y data\\n-y data\\neutron\\' + str(i+1) + '.csv'))
    for i in range(d2):
        data.append(get_data(r'D:\\My Files\\核技术专题研讨\\n-y data\\n-y data\\gamma\\' + str(i+1) + '.csv'))
    data1 = data[:d1]
    data2 = data[d1:]

    indata = torch.empty(l1+l2, din)
    for i in range(l1):
        for j in range(din):
            indata[i, j] = data1[i][j]
    for i in range(l2):
        for j in range(din):
            indata[i+l1, j] = data2[i][j]
    
    outdata = torch.empty(l1+l2, dout)
    for i in range(l1):
        outdata[i, 0] = 0
        outdata[i, 1] = 1
    for i in range(l2):
        outdata[i+l1, 0] = 1
        outdata[i+l1, 1] = 0

    checkdata = torch.tensor(data1[l1:] + data2[l2:])
    label = torch.tensor([[0, 1] for i in range(len(data1[l1:]))] + [[1, 0] for i in range(len(data2[l2:]))])

    return [indata, outdata, checkdata, label]

Check model

In [5]:
def check(model, din, dout):
    checkdata = read_data(din, dout, 33, 28, 20, 20)[2]
    label = read_data(din, dout, 33, 28, 20, 20)[3]
    lossfunc = torch.nn.MSELoss()
    loss = lossfunc(model(checkdata), label)
    print('loss = ', loss.item())

    result = []
    correct = 0
    for i in range(len(checkdata)):
        if (model(checkdata)[i][0] > 0.5 and model(checkdata)[i][1] < 0.5) or (model(checkdata)[i][0] > model(checkdata)[i][1]):
            result.append([1, 0])
        elif (model(checkdata)[i][1] > 0.5 and model(checkdata)[i][0] < 0.5) or (model(checkdata)[i][1] > model(checkdata)[i][0]):
            result.append([0, 1])
        print(i+1, model(checkdata)[i], result[i], list(label[i]))
        if result[i] == list(label[i]):
            correct += 1
    corate = correct / len(checkdata)
    with open('accu.txt', 'a') as f:
        print('Accuracy = ', 100*corate, '%', file=f)

Parameters

In [5]:
N, din, H, dout, it = 40, 252, 100, 2, 20000
indata = read_data(din, dout, 33, 28, 20, 20)[0]
outdata = read_data(din, dout, 33, 28, 20, 20)[1]

Define model, loss function and optimizer

In [6]:
model = torch.nn.Sequential(
    torch.nn.Linear(din, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, dout),
)

lossfunc = torch.nn.MSELoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## Training

In [7]:
for j in [10, 20, 50, 100, 200, 300, 400, 500, 1000, 2000, 5000, 10000, 15000]:
    for i in range(j):
        predy = model(indata)

        loss = lossfunc(predy, outdata)
        print(i+1, loss.item())

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        
    check(model, din, dout)

 1.7881096333016407e-15
14358 4.334502219226644e-15
14359 4.074095492120466e-15
14360 2.963395587946005e-15
14361 3.1166149542583618e-15
14362 3.67660834096244e-15
14363 2.583688928900268e-15
14364 3.838501324654681e-15
14365 3.0273878683198833e-15
14366 2.3045286805836425e-15
14367 3.903553455003811e-15
14368 4.701829915264733e-15
14369 2.640119110879297e-15
14370 4.876264492290495e-15
14371 6.005655872512016e-15
14372 3.652907512065318e-15
14373 4.408366033326061e-15
14374 4.909625308434579e-15
14375 1.739615938280148e-15
14376 6.030085996776725e-15
14377 6.786368257578673e-15
14378 2.111505330255675e-15
14379 6.67330757032859e-15
14380 7.009250578088539e-15
14381 2.1946528372152356e-15
14382 4.6842844747953076e-15
14383 4.1994106285349504e-15
14384 2.020532721171142e-15
14385 3.624653034043756e-15
14386 2.7625606880202214e-15
14387 3.0704957043462596e-15
14388 5.70615603379121e-15
14389 5.749949754746625e-15
14390 2.7487697210895005e-15
14391 4.752114873211432e-15
14392 4.5830661551

In [9]:
import time
tstart = time.time()
testdata = []
testdata_norm = []

for i in range(10):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0000' + str(i) + '.csv'))
for i in range(90):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--000' + str(i+10) + '.csv'))
for i in range(900):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--00' + str(i+100) + '.csv'))
for i in range(9000):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0' + str(i+1000) + '.csv'))
for i in range(9243):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--' + str(i+10000) + '.csv'))

for i in range(len(testdata)):
    testdata_norm.append(normalize(testdata[i]))
    writedata(testdata_norm[i], str(i) + '.txt')

testdata = torch.tensor(testdata)
tend = time.time()
print(tend - tstart)

41.18225455284119


In [ ]:
result = []
gamma = 0
neutron = 0
output = model(testdata)

Data processing and output

In [ ]:
for i in range(len(testdata)):
    a = output[i][0].item()
    b = output[i][1].item()
    if a > b:
        result.append([1, 0])
        par = 'gamma'
        gamma += 1
    elif b > a:
        result.append([0, 1])
        par = 'neutron'
        neutron += 1
    with open('result.txt', 'a') as f:
        #print(i+1, a, b, result[i])
        print('file = ' + str(i), 'particle =', par, file=f)
    with open('stat.txt', 'a') as f:
        print(b, file=f)
with open('result.txt', 'a') as f:
    print('total gamma =', gamma, 'total neutron =', neutron, file=f)

In [7]:
def normalize(signal):
    peak = min(signal)
    norm = signal
    for i in range(len(signal)):
        norm[i] = signal[i]/peak
    return norm

def writedata(signal, filename):
    with open('norm/' + filename, 'a') as f:
        for i in range(len(signal)):
            print(signal[i], file = f)

In [11]:
import shutil
for i in range(192):
    shutil.copyfile('norm/' + str(100*i) + '.txt', 'sample/' + str(10*i) + '.txt')